In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import numpy as np
from sklearn.preprocessing import StandardScaler


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
data = pd.read_csv('facies_vectors.csv')
data = data.fillna(data['PE'].mean())
feature_names = ['GR_diff_up', 'ILD_log10_diff_up', 'DeltaPHI_diff_up', 'PHIND_diff_up', 'PE_diff_up', 'NM_M_diff_up', 'RELPOS_diff_up','GR_diff_down', 'ILD_log10_diff_down', 'DeltaPHI_diff_down', 'PHIND_diff_down', 'PE_diff_down', 'NM_M_diff_down', 'RELPOS_diff_down','GR', 'ILD_log10', 'DeltaPHI', 'PHIND', 'PE', 'NM_M', 'RELPOS']
feature_names2 = ['GR', 'ILD_log10', 'DeltaPHI', 'PHIND', 'PE', 'NM_M', 'RELPOS']
feature_names3 = ['GR', 'ILD_log10', 'DeltaPHI', 'PHIND', 'PE', 'NM_M', 'RELPOS','GR_diff_up', 'ILD_log10_diff_up', 'DeltaPHI_diff_up', 'PHIND_diff_up', 'PE_diff_up', 'NM_M_diff_up', 'RELPOS_diff_up']
feature_names4 = ['GR_diff_up', 'ILD_log10_diff_up', 'DeltaPHI_diff_up', 'PHIND_diff_up', 'PE_diff_up', 'NM_M_diff_up', 'RELPOS_diff_up','GR_diff_down', 'ILD_log10_diff_down', 'DeltaPHI_diff_down', 'PHIND_diff_down', 'PE_diff_down', 'NM_M_diff_down', 'RELPOS_diff_down']
facies_names = ['SS', 'CSiS', 'FSiS', 'SiSh', 'MS', 'WS', 'D', 'PS', 'BS']
facies_colors = ['#F4D03F', '#F5B041','#DC7633','#6E2C00', '#1B4F72','#2E86C1', '#AED6F1', '#A569BD', '#196F3D']

In [4]:
[set(data['Facies'][data['Well Name'] == well]) for well in set(data['Well Name'])]


[{1, 2, 3, 4, 5, 6, 7, 8},
 {2, 3, 4, 5, 6, 7, 8, 9},
 {2, 3, 4, 5, 6, 7, 8, 9},
 {1, 2, 3, 4, 5, 6, 7, 8},
 {1, 2, 3, 4, 5, 6, 7, 8, 9},
 {1, 2, 3, 4, 5, 6, 7, 8},
 {9},
 {1, 2, 3, 4, 5, 6, 7, 8, 9},
 {2, 3, 4, 5, 6, 7, 8},
 {2, 3, 4, 5, 6, 7, 8, 9}]

In [5]:
[(face,len(data[data['Facies'] == face])) for face in set(data['Facies'])]

[(1, 268),
 (2, 940),
 (3, 780),
 (4, 271),
 (5, 296),
 (6, 582),
 (7, 141),
 (8, 686),
 (9, 185)]

In [6]:
[(well,len(data[data['Well Name'] == well])) for well in set(data['Well Name'])]

[('CROSS H CATTLE', 501),
 ('ALEXANDER D', 466),
 ('NEWBY', 463),
 ('NOLAN', 415),
 ('KIMZEY A', 439),
 ('SHANKLE', 449),
 ('Recruit F9', 80),
 ('CHURCHMAN BIBLE', 404),
 ('LUKE G U', 461),
 ('SHRIMPLIN', 471)]

In [7]:
for well in set(data['Well Name']):
    data_well = data[data['Well Name'] == well]            
    print([(well, face, len(data_well[data_well['Facies'] == face])) for face in set(data_well['Facies'])])

[('CROSS H CATTLE', 1, 158), ('CROSS H CATTLE', 2, 142), ('CROSS H CATTLE', 3, 47), ('CROSS H CATTLE', 4, 25), ('CROSS H CATTLE', 5, 28), ('CROSS H CATTLE', 6, 31), ('CROSS H CATTLE', 7, 2), ('CROSS H CATTLE', 8, 68)]
[('ALEXANDER D', 2, 117), ('ALEXANDER D', 3, 91), ('ALEXANDER D', 4, 44), ('ALEXANDER D', 5, 26), ('ALEXANDER D', 6, 69), ('ALEXANDER D', 7, 16), ('ALEXANDER D', 8, 98), ('ALEXANDER D', 9, 5)]
[('NEWBY', 2, 98), ('NEWBY', 3, 80), ('NEWBY', 4, 58), ('NEWBY', 5, 28), ('NEWBY', 6, 96), ('NEWBY', 7, 16), ('NEWBY', 8, 56), ('NEWBY', 9, 31)]
[('NOLAN', 1, 4), ('NOLAN', 2, 118), ('NOLAN', 3, 68), ('NOLAN', 4, 28), ('NOLAN', 5, 47), ('NOLAN', 6, 30), ('NOLAN', 7, 4), ('NOLAN', 8, 116)]
[('KIMZEY A', 1, 9), ('KIMZEY A', 2, 85), ('KIMZEY A', 3, 74), ('KIMZEY A', 4, 43), ('KIMZEY A', 5, 53), ('KIMZEY A', 6, 51), ('KIMZEY A', 7, 27), ('KIMZEY A', 8, 90), ('KIMZEY A', 9, 7)]
[('SHANKLE', 1, 89), ('SHANKLE', 2, 89), ('SHANKLE', 3, 117), ('SHANKLE', 4, 7), ('SHANKLE', 5, 19), ('SHANKLE'

In [8]:
def find_diff(row, well):
    if len(prev_depth_features[well]) == 0:
        prev_depth_features[well] = row.values[4:]
        return
    diff = (row.values[4:] - prev_depth_features[well])/0.5
    prev_depth_features[well] = row.values[4:]
    return diff
data_well = dict()
data_well_inverse = dict()
prev_depth_features = dict()
new_data = pd.DataFrame()
prev_class= dict()
data_save = pd.DataFrame()
for well in set(data['Well Name']):
    prev_depth_features[well] = []
    prev_class[well] = []
    data_well[well] = data[data['Well Name'] == well]
    data_well[well] = data_well[well].sort_values(by=['Depth'])
    data_save = data_well[well].iloc[::-1]
    data_well[well]['diff_up'] = data_well[well].apply(lambda row: find_diff(row, well), axis=1)
    prev_depth_features[well] = []
    prev_class[well] = []
    ##
    data_save = data_save.apply(lambda row: find_diff(row, well), axis=1)
    data_well[well]['diff_down'] = data_save.iloc[::-1]
    data_well[well] = data_well[well].dropna()
    data_well[well]['GR_diff_up'] = data_well[well].apply(lambda row: row['diff_up'][0], axis=1)
    data_well[well]['ILD_log10_diff_up'] = data_well[well].apply(lambda row: row['diff_up'][1], axis=1)
    data_well[well]['DeltaPHI_diff_up'] = data_well[well].apply(lambda row: row['diff_up'][2], axis=1)
    data_well[well]['PHIND_diff_up'] = data_well[well].apply(lambda row: row['diff_up'][3], axis=1)
    data_well[well]['PE_diff_up'] = data_well[well].apply(lambda row: row['diff_up'][4], axis=1)
    data_well[well]['NM_M_diff_up'] = data_well[well].apply(lambda row: row['diff_up'][5], axis=1)
    data_well[well]['RELPOS_diff_up'] = data_well[well].apply(lambda row: row['diff_up'][6], axis=1)
    #######
    data_well[well]['GR_diff_down'] = data_well[well].apply(lambda row: row['diff_down'][0], axis=1)
    data_well[well]['ILD_log10_diff_down'] = data_well[well].apply(lambda row: row['diff_down'][1], axis=1)
    data_well[well]['DeltaPHI_diff_down'] = data_well[well].apply(lambda row: row['diff_down'][2], axis=1)
    data_well[well]['PHIND_diff_down'] = data_well[well].apply(lambda row: row['diff_down'][3], axis=1)
    data_well[well]['PE_diff_down'] = data_well[well].apply(lambda row: row['diff_down'][4], axis=1)
    data_well[well]['NM_M_diff_down'] = data_well[well].apply(lambda row: row['diff_down'][5], axis=1)
    data_well[well]['RELPOS_diff_down'] = data_well[well].apply(lambda row: row['diff_down'][6], axis=1)
    new_data = pd.concat([new_data, data_well[well]])
    new_data = new_data.drop(['diff_up'], axis=1)
    #
    new_data = new_data.drop(['diff_down'], axis=1)

In [9]:
set(new_data['Well Name'])

{'ALEXANDER D',
 'CHURCHMAN BIBLE',
 'CROSS H CATTLE',
 'KIMZEY A',
 'LUKE G U',
 'NEWBY',
 'NOLAN',
 'Recruit F9',
 'SHANKLE',
 'SHRIMPLIN'}

In [15]:
def augment_features_window(X, N_neig):
    N_row = X.shape[0]
    N_feat = X.shape[1]
    X = np.vstack((np.zeros((N_neig, N_feat)), X, np.zeros((N_neig, N_feat))))
    X_aug = np.zeros((N_row, N_feat*(2*N_neig+1)))
    for r in np.arange(N_row) + N_neig:
        this_row = []
        for c in np.arange(-N_neig,N_neig+1):
            #this_row = np.hstack((this_row, X[r+c]))
            #if c != 0:
                #print(len((X[r] + X[r+c])/2))
            this_row = np.hstack((this_row, (X[r] + X[r+c])/2))
        #print(len(this_row))
        X_aug[r-N_neig] = this_row

    return X_aug

def augment_features_gradient(X, depth):
    d_diff = np.diff(depth).reshape((-1, 1))
    d_diff[d_diff==0] = 0.001
    X_diff = np.diff(X, axis=0)
    X_grad = X_diff / d_diff
    X_grad = np.concatenate((X_grad, np.zeros((1, X_grad.shape[1]))))
    
    return X_grad

def augment_features(X, well, depth, N_neig=2):
    X_aug = np.zeros((X.shape[0], X.shape[1]*(2*N_neig+1)))
    for w in np.unique(well):
        w_idx = np.where(well == w)[0]
        X_aug_win = augment_features_window(X[w_idx, :], N_neig)
        #print(X_aug_win)
        X_aug_grad = augment_features_gradient(X[w_idx, :], depth[w_idx])
        #print(X_aug_grad)
#         X_aug[w_idx, :] = X_aug_win
        X_aug[w_idx, :] = np.concatenate((X_aug_win, X_aug_grad), axis=1)
        
    return X_aug


In [16]:
def accuracy(y_predict, y_test):
    truth = [y_predict[index] for index in range(len(y_predict)) if y_predict[index] == y_test[index]]
    return len(truth)/len(y_predict)

In [17]:
import xgboost as xgb
from xgboost.sklearn import  XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import f1_score
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB
def classification(clf):
    acc = 0
    clf.fit(X_train_robust_norm , y_train)
    y_predict = clf.predict(X_test_robust_norm)
    acc += f1_score(y_test, y_predict, average='micro')
    print('With augm',acc)
    return acc

In [18]:
def X_to_float(X):
    for row in range(len(X)):
        for column in range(len(X[0])):
            X[row, column] = float(X[row, column])
    return X

In [19]:
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
global_acc = 0
global_acc_f1 = 0
global_acc3 = 0
test = dict()
train = dict()
index = 0
for well in set(data['Well Name']):

    test[well] = new_data[new_data['Well Name'] == well]
    train[well] = new_data[new_data['Well Name'] != well]
    X_train_1 = train[well][feature_names].values 
#     X_train_2 = train[well][feature_names4].values 
    y_train = train[well]['Facies'].values 
    X_test_1 = test[well][feature_names].values 
#     X_test_2 = test[well][feature_names4].values 
    y_test = test[well]['Facies'].values 
    well_train = train[well]['Well Name'].values
    well_test = test[well]['Well Name'].values
    depth_train = train[well]['Depth'].values
    depth_test = test[well]['Depth'].values    

    X_aug_train = augment_features(X_train_1,well_train,depth_train)
    X_aug_test = augment_features(X_test_1,well_test,depth_test)
    #print(len(X_aug_train[0]))
    #print(len(X_train_1[0]))
    X_train = np.concatenate((X_aug_train, X_train_1), axis=1)
    X_test = np.concatenate((X_aug_test, X_test_1), axis=1)
    robust = preprocessing.RobustScaler(quantile_range=(25.0, 75.0)).fit(X_train)
    X_train_robust = robust.transform(X_train)
    X_test_robust = robust.transform(X_test)

    scaler = StandardScaler().fit(X_train_robust)
    X_train_robust_norm = scaler.transform(X_train_robust)
    X_test_robust_norm = scaler.transform(X_test_robust)

    print(well)
    clf_xgb = XGBClassifier(max_depth=3, learning_rate=0.12, n_estimators=150)
    global_acc += classification(clf_xgb)
    
print(global_acc/10)

ValueError: shape mismatch: value array of shape (464,126) could not be broadcast to indexing result of shape (464,105)

In [ ]:
a = (1, 2, 3)
b =(4, 5, 6)
c = np.concatenate((a, b), axis=0)
c

In [ ]:
clf_xgb = XGBClassifier(learning_rate=0.12, max_depth=3,n_estimators=150)
classification(clf_xgb)

In [1]:
clf_log = LogisticRegression(multi_class='ovr')
classification(clf_log)


NameError: name 'LogisticRegression' is not defined

In [11]:
clf_per = MLPClassifier(hidden_layer_sizes=(100,100,100))
classification(clf_per)

With augm 0.5094967532467533
without anything 0.5349397590361445
With robust and aug 0.528814935064935
With robust 0.7278212525426381
With robust and aug and norm 0.5387175324675325
With robust and norm 0.7187713516403276


In [15]:
from keras.utils import to_categorical
y_train_tocat = to_categorical(y_train, 10)
y_test_tocat = to_categorical(y_test, 10)
model = Sequential()
model.add(Dense(100, input_dim=7, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_robust_norm, y_train_tocat, validation_split=0.3, epochs=200, batch_size=10)
model.evaluate(X_test_robust_norm, y_test_tocat)

Train on 2032 samples, validate on 872 samples
Epoch 1/200
2032/2032 [==============================] - 1s 602us/step - loss: 1.4418 - acc: 0.4419 - val_loss: 1.1382 - val_acc: 0.5321
Epoch 2/200
2032/2032 [==============================] - 1s 401us/step - loss: 1.0839 - acc: 0.5595 - val_loss: 1.0559 - val_acc: 0.5757
Epoch 3/200
2032/2032 [==============================] - 1s 399us/step - loss: 1.0204 - acc: 0.5738 - val_loss: 1.0351 - val_acc: 0.5872
Epoch 4/200
2032/2032 [==============================] - 1s 414us/step - loss: 0.9801 - acc: 0.5984 - val_loss: 1.0218 - val_acc: 0.5883
Epoch 5/200
2032/2032 [==============================] - 1s 412us/step - loss: 0.9572 - acc: 0.6038 - val_loss: 0.9976 - val_acc: 0.5975
Epoch 6/200
2032/2032 [==============================] - 1s 405us/step - loss: 0.9268 - acc: 0.6147 - val_loss: 0.9828 - val_acc: 0.6170
Epoch 7/200
2032/2032 [==============================] - 1s 421us/step - loss: 0.9095 - acc: 0.6289 - val_loss: 0.9733 - val_acc: 0

2032/2032 [==============================] - 1s 414us/step - loss: 0.3608 - acc: 0.8469 - val_loss: 1.1219 - val_acc: 0.6640
Epoch 61/200
2032/2032 [==============================] - 1s 400us/step - loss: 0.3622 - acc: 0.8538 - val_loss: 1.1036 - val_acc: 0.6743
Epoch 62/200
2032/2032 [==============================] - 1s 409us/step - loss: 0.3439 - acc: 0.8647 - val_loss: 1.0935 - val_acc: 0.6766
Epoch 63/200
2032/2032 [==============================] - 1s 405us/step - loss: 0.3407 - acc: 0.8612 - val_loss: 1.1531 - val_acc: 0.6835
Epoch 64/200
2032/2032 [==============================] - 1s 399us/step - loss: 0.3356 - acc: 0.8602 - val_loss: 1.1496 - val_acc: 0.6823
Epoch 65/200
2032/2032 [==============================] - 1s 376us/step - loss: 0.3287 - acc: 0.8696 - val_loss: 1.1381 - val_acc: 0.6892
Epoch 66/200
2032/2032 [==============================] - 1s 402us/step - loss: 0.3317 - acc: 0.8745 - val_loss: 1.1298 - val_acc: 0.6881
Epoch 67/200
2032/2032 [=======================

2032/2032 [==============================] - 1s 400us/step - loss: 0.1074 - acc: 0.9675 - val_loss: 2.1094 - val_acc: 0.6789
Epoch 179/200
2032/2032 [==============================] - 1s 429us/step - loss: 0.1677 - acc: 0.9562 - val_loss: 2.1435 - val_acc: 0.6743
Epoch 180/200
2032/2032 [==============================] - 1s 412us/step - loss: 0.2293 - acc: 0.9291 - val_loss: 2.0666 - val_acc: 0.6904
Epoch 181/200
2032/2032 [==============================] - 1s 414us/step - loss: 0.1123 - acc: 0.9656 - val_loss: 2.1346 - val_acc: 0.6950
Epoch 182/200
2032/2032 [==============================] - 1s 409us/step - loss: 0.1107 - acc: 0.9636 - val_loss: 2.1475 - val_acc: 0.6709
Epoch 183/200
2032/2032 [==============================] - 1s 421us/step - loss: 0.0865 - acc: 0.9754 - val_loss: 2.0536 - val_acc: 0.6915
Epoch 184/200
2032/2032 [==============================] - 1s 411us/step - loss: 0.0681 - acc: 0.9808 - val_loss: 2.0516 - val_acc: 0.6858
Epoch 185/200
2032/2032 [================

[1.9645717413071169, 0.6899598386871767]

In [16]:
import pandas as pd
df = pd.DataFrame()
df['y1'] = [1, 2]
df['y2'] = [2, 3]
df

,y1,y2
0,1,2
1,2,3


In [45]:
import numpy as np
def add_std(y1, y2):
    return np.std([y1, y2]), np.mean([y1, y2]), min([y1, y2])
df['Result'] = df.apply(lambda row: add_std(row['y1'], row['y2']), axis=1)

In [50]:
df['std'] = df.apply(lambda row: row['Result'][0], axis = 1)
df['mean'] = df.apply(lambda row: row['Result'][1], axis = 1)
df['min'] = df.apply(lambda row: row['Result'][2], axis = 1)

In [56]:
df

,y1,y2,std,mean,Result,min
0,1,2,0.5,1.5,"(0.5, 1.5, 1.0)",1.0
1,2,3,0.5,2.5,"(0.5, 2.5, 2.0)",2.0


In [58]:
df = df.drop(['Result'], axis=1)

In [59]:
df

,y1,y2,std,mean,min
0,1,2,0.5,1.5,1.0
1,2,3,0.5,2.5,2.0


In [ ]:
acc= 0
    for iter in range(10):
        clf.fit(X_train, y_train)
        y_predict = clf.predict(X_test)
        acc += accuracy(y_predict, y_test)
    acc /= 10
    print('without anything',acc)
    acc= 0
    for iter in range(10):
        clf.fit(X_aug_train_robust, y_aug_train)
        y_predict = clf.predict(X_aug_test_robust)
        acc += accuracy(y_predict, y_aug_test)
    acc /= 10
    print('With robust and aug',acc)
    acc = 0
    for iter in range(10):
        clf.fit(X_train_robust, y_train)
        y_predict = clf.predict(X_test_robust)
        acc += accuracy(y_predict, y_test)
    acc /= 10
    print('With robust',acc)
    acc= 0
    for iter in range(10):
        clf.fit(X_aug_train_robust_norm, y_aug_train)
        y_predict = clf.predict(X_aug_test_robust_norm)
        acc += accuracy(y_predict, y_aug_test)
    acc /= 10
    print('With robust and aug and norm',acc)
    acc = 0
    for iter in range(10):
        clf.fit(X_train_robust_norm, y_train)
        y_predict = clf.predict(X_test_robust_norm)
        acc += accuracy(y_predict, y_test)
    acc /= 10
    print('With robust and norm',acc)

In [ ]:
robust = preprocessing.RobustScaler(quantile_range=(25.0, 75.0)).fit(X_aug_train)
    X_aug_train_robust = robust.transform(X_aug_train)
    X_aug_test_robust = robust.transform(X_aug_test)

    robust = preprocessing.RobustScaler(quantile_range=(25.0, 75.0)).fit(X_train)
    X_train_robust = robust.transform(X_train)
    X_test_robust = robust.transform(X_test)

    scaler = StandardScaler().fit(X_aug_train_robust)
    X_aug_train_robust_norm = scaler.transform(X_aug_train_robust)
    X_aug_test_robust_norm = scaler.transform(X_aug_test_robust)

    scaler = preprocessing.RobustScaler(quantile_range=(25.0, 75.0)).fit(X_train_robust)
    X_train_robust_norm = scaler.transform(X_train_robust)
    X_test_robust_norm = scaler.transform(X_test_robust)